In [ ]:
!pip install crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.7/308.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.5/583.5 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.6 

In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 8.7 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
import os

In [ ]:
os.environ["TAVILY_API_KEY"] = ""

In [ ]:
os.environ["HUGGINGFACE_API_KEY"] = ""

In [ ]:
os.environ["GROQ_API_KEY"]=""

In [ ]:
from crewai import Agent,Task,Crew
from langchain.retrievers import TavilySearchAPIRetriever
from langchain.tools import Tool

## LLM

In [ ]:
from langchain_groq import ChatGroq
from crewai import LLM

In [ ]:
llm = ChatGroq(
            temperature=0.7,
            model="groq/llama3-8b-8192",
            groq_api_key=os.getenv("GROQ_API_KEY"),
            max_tokens=500
        )

In [ ]:
#llm = LLM(model="huggingface/meta-llama/Llama-3.1-8B-Instruct")

## Tools

In [ ]:
from crewai.tools import BaseTool
from pydantic import PrivateAttr

In [ ]:
from crewai import Agent
from langchain.retrievers import TavilySearchAPIRetriever

In [ ]:
search=TavilySearchAPIRetriever(
            api_key=os.getenv('TAVILY_API_KEY'),
            k=6
        )

In [ ]:
from pydantic import BaseModel,Field

In [ ]:
class ResearchToolInput(BaseModel):
    topic: str = Field(..., description="The topic to search for. This must be a plain string.")

class VideoToolInput(BaseModel):
    topic: str

class WriterToolInput(BaseModel):
    research_brief: str


In [ ]:
class ResearchTool(BaseTool):
    name: str = "Research Topic"
    description: str = "Searches for information about a given topic using Tavily"
    args_schema : Type[BaseModel] = ResearchToolInput
    _searcher: TavilySearchAPIRetriever = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._searcher = search

    def _run(self, topic: str) -> str:
        if not isinstance(topic, str):
            return "Error: 'topic' must be a plain string."
        try:
            search_results = self._searcher.invoke(topic)

            research_summary = "Research findings:\n\n"
            for i, doc in enumerate(search_results, 1):
                research_summary += f"{i}. {doc.page_content}\n\n"

            research_summary += "\nSources:\n"
            for i, doc in enumerate(search_results, 1):
                if doc.metadata.get('source'):
                    research_summary += f"- {doc.metadata['source']}\n"

            return research_summary
        except Exception as e:
            return f"Error during research: {str(e)}"

In [ ]:
from typing import Type

In [ ]:
class VideoSearchTool(BaseTool):
    name: str = "Search Videos"
    description: str = "Searches for YouTube videos related to a topic"
    args_schema: Type[BaseModel] = VideoToolInput
    _searcher: TavilySearchAPIRetriever = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._searcher = TavilySearchAPIRetriever(k=5)

    def _run(self, topic: str) -> str:
        try:
            query = f"site:youtube.com {topic}"
            results = self._searcher.invoke(query)

            videos = []
            for doc in results:
                title = doc.metadata.get("title", "Untitled")
                url = doc.metadata.get("source", "No URL")
                videos.append(f"{title} - {url}")

            return "\n".join(videos) if videos else "No videos found."
        except Exception as e:
            return f"Error during video search: {str(e)}"



In [ ]:
class WriterTool(BaseTool):
    name: str = "Write LinkedIn Post"
    description: str = "Creates an engaging LinkedIn post from research findings"
    args_schema = WriterToolInput
    _llm: ChatGroq = PrivateAttr()

    def __init__(self):
        super().__init__()
        self._llm = llm

    def _run(self, research_brief: str) -> str:
        if not isinstance(research_brief, str):
            return "Error: 'research_brief' must be a string."

        prompt = f"""Create a professional LinkedIn post based on this research:
{research_brief}

Follow these guidelines:
- Write for a professional audience on LinkedIn
- Use an engaging hook in the first line
- Keep it under 1300 characters
- Use short paragraphs and line breaks for readability
- Include relevant emojis (but not too many)
- Use 3 to 5 relevant hashtags at the end
- Include a call to action (question, invitation, or suggestion)
- Make it professional and thought-provoking"""

        try:
            response = self._llm.invoke(prompt)
            return response.content
        except Exception as e:
            return f"Error while generating LinkedIn post: {str(e)}"

In [ ]:
research_tool=ResearchTool()

In [ ]:
print(research_tool._run("what is llm"))

Research findings:

1. A large language model (LLM) is a type of machine learning model designed for natural language processing tasks such as language generation. LLMs are language models with many parameters, and are trained with self-supervised learning on a vast amount of text. A smoothed n-gram model in 2001 trained on 0.3 billion words achieved state-of-the-art perplexity at the time.[4] In the 2000s, as Internet use became prevalent, some researchers constructed Internet-scale language datasets ("web as corpus"[5]), upon which they trained statistical language models.[6][7] In 2009, in most language processing tasks, statistical language models dominated over symbolic language models because they can usefully ingest large datasets.[8]

2. A *large language model* is a type of artificial intelligence algorithm that applies neural network techniques with lots of parameters to process and understand human languages or text using self-supervised learning techniques. Fine Tuning Larg

In [ ]:
video_tool=VideoSearchTool()

In [ ]:
print(video_tool._run("what is llm"))

What are Large Language Models (LLMs)? - YouTube - https://www.youtube.com/watch?v=iR2O2GPbB0E
LLM Explained Simply | What is LLM? - YouTube - https://www.youtube.com/watch?v=uTSGHBsvplg
LLM Explained | What is LLM - YouTube - https://www.youtube.com/watch?v=67_aMPDk2zw
What is LLM (Large Language Model) | How Large Language ... - YouTube - https://www.youtube.com/watch?v=fn9T5yPyv1o
What is an LLM? | How Large Language Models Work (Complete ... - YouTube - https://www.youtube.com/watch?v=NBbJ7rVv9RM


In [ ]:
writing_tool=WriterTool()

## Agents

In [ ]:
researcher = Agent(
            role='Research Analyst',
            goal=(
              "Research trending topics and create content briefs. "
              "When using the 'Research Topic' tool, always pass the input as: "
              '{"topic": "the exact topic as a plain string"}. '
              "Never include nested dictionaries or fields like 'description' or 'type'."
            ),
            backstory="""You are a precise researcher. You must follow input format rules strictly.
            When invoking the 'Research Topic' tool, make sure the Action Input looks like:
            {"topic": "example"} — never include keys like 'description' or 'type'.
            """,
            llm=llm,
            tools=[research_tool],
            verbose=True
        )

In [ ]:
video_agent = Agent(
    role="Video Researcher",
    goal=(
        "Search for relevant YouTube videos related to a topic. "
        "Use the 'Search Videos' tool and provide the Action Input in this exact format: "
        "{\"topic\": \"your video topic\"}. Do not include fields like 'description' or 'type'."
    ),
    backstory=(
        "You are a content research assistant specialized in sourcing videos. "
        "Your Action Input must always be a clean JSON object with one string field: "
        "{\"topic\": \"...\"}. Avoid nesting or meta-descriptions."
    ),
    llm=llm,
    tools=[video_tool],
    verbose=True
)


In [ ]:
writer = Agent(
    role='Content Writer',
    goal=(
        "Create a professional LinkedIn post from a plain text research summary. "
        "Use only the 'Write LinkedIn Post' tool. The input 'research_brief' must be a single clean string. "
        "Do NOT wrap it inside objects or lists. Use the content as-is to write the post."
    ),
    backstory=(
        "You are a top-tier LinkedIn content creator. Your job is to write powerful posts using flat string input. "
        "NEVER use structured JSON objects like {'description': ..., 'type': ...}. Your only input is a plain string."
    ),
    llm=llm,
    tools=[writing_tool],
    verbose=True
)


In [ ]:
topic = input("Enter the topic for your LinkedIn post: ")

Enter the topic for your LinkedIn post: what is llm


In [ ]:
topic

'what is llm'

In [ ]:
research_task = Task(
            description=f"Research this topic thoroughly: {topic}",
            agent=researcher,
            expected_output="Detailed research findings including key insights, trends, and statistics about the topic",
            input=topic,
            output_key="research_task"
        )

In [ ]:
video_task = Task(
    description=f"Find videos related to the topic: {topic}",
    agent=video_agent,
    expected_output="A list of relevant video links related to the topic (e.g., YouTube)",
    input=topic,
    output_key="video_links"
)


In [ ]:
writing_task = Task(
    description="Create an engaging LinkedIn post from research and include selected videos",
    agent=writer,
    expected_output="A LinkedIn post with engaging content, emojis, hashtags, and embedded videos links",
    input={
        "research_brief": "{{ research_task.output }}",
        "videos": "{{ video_links }}"
    }
)

In [ ]:
content_crew = Crew(
    agents=[researcher,video_agent ,writer],
    tasks=[research_task,video_task ,writing_task]
)

result = content_crew.kickoff()